<a href="https://colab.research.google.com/github/Zfeng0207/FIT3199-FYP/blob/dev%2Fdarren/Darren_NB_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
import gdown

import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


In [2]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FIT3199-FYP/input/01_preprocessed_output.csv')
data

,subject_id,stay_id_x,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,...,seq_num,icd_code,icd_version,icd_title,Stroke_Y/N,gender,anchor_age,anchor_year,anchor_year_group,dod
0,14394983,34259230.0,2202-07-10 12:24:00,98.3,74.0,19.0,97.0,153.0,67.0,-1,...,2,262,10,369,0,1,46,2196,0,NaN
1,17730806,30258441.0,2182-07-17 16:58:00,NaN,82.0,16.0,96.0,115.0,102.0,-1,...,1,58,9,256,1,1,68,2176,1,NaN
2,15973805,30481231.0,2145-07-03 17:20:00,98.0,109.0,18.0,98.0,113.0,77.0,-1,...,2,471,9,452,0,0,51,2139,1,NaN
3,16945691,36369419.0,2117-12-18 23:50:00,99.9,74.0,16.0,100.0,151.0,76.0,-1,...,1,388,10,158,0,0,81,2111,1,2118-09-20
4,15632719,33618002.0,2170-04-19 20:58:00,98.4,83.0,NaN,94.0,142.0,81.0,-1,...,1,58,9,256,1,1,55,2170,2,2170-10-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52285,15853169,33648217.0,2174-04-13 17:56:00,98.1,62.0,16.0,100.0,121.0,62.0,-1,...,2,330,10,482,1,0,52,2170,2,NaN
52286,10913302,37712176.0,2196-11-26 09:23:00,NaN,NaN,NaN,NaN,NaN,NaN,-1,...,1,403,10,203,0,1,25,2185,0,NaN
52287,11553072,31817224.0,2164-12-23 16:45:00,99.8,92.0,16.0,96.0,160.0,80.0,-1,...,1,34,9,34,0,1,46,2156,0,NaN
52288,10295929,35519857.0,2150-12-05 06:15:00,97.9,63.0,20.0,99.0,149.0,97.0,-1,...,3,462,9,219,1,1,61,2141,0,NaN


In [3]:
print(data.columns)

Index(['subject_id', 'stay_id_x', 'charttime', 'temperature', 'heartrate',
       'resprate', 'o2sat', 'sbp', 'dbp', 'rhythm', 'pain', 'stay_id_y',
       'seq_num', 'icd_code', 'icd_version', 'icd_title', 'Stroke_Y/N',
       'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod'],
      dtype='object')


In [16]:
# Drop non-relevant columns
columns_to_drop = ["subject_id", "stay_id_x", "stay_id_y", "charttime", "dod", "icd_title"]
data = data.drop(columns=columns_to_drop, errors='ignore')

# Handle missing values by filling with column mean
data = data.fillna(data.mean())

# Define target variable
target_column = "Stroke_Y/N"
X = data.drop(columns=[target_column]).values
y = data[target_column].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Split dataset into train, validation, and test sets
train_size = int(0.7 * len(X_tensor))
val_size = int(0.15 * len(X_tensor))
test_size = len(X_tensor) - train_size - val_size
train_data, val_data, test_data = random_split(TensorDataset(X_tensor, y_tensor), [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Train Naive Bayes Model
def train_naive_bayes(train_loader, val_loader):
    X_train, y_train = [], []
    for X_batch, y_batch in train_loader:
        X_train.append(X_batch.numpy())
        y_train.append(y_batch.numpy())
    X_train = np.vstack(X_train)
    y_train = np.hstack(y_train)

    model = GaussianNB()
    model.fit(X_train, y_train)

    # Validation Accuracy
    X_val, y_val = [], []
    for X_batch, y_batch in val_loader:
        X_val.append(X_batch.numpy())
        y_val.append(y_batch.numpy())
    X_val = np.vstack(X_val)
    y_val = np.hstack(y_val)

    y_val_pred = model.predict(X_val)
    val_acc = accuracy_score(y_val, y_val_pred)
    print(f'Validation Accuracy: {val_acc:.4f}')
    return model

nb_model = train_naive_bayes(train_loader, val_loader)

# Evaluate Naive Bayes Model
def evaluate_naive_bayes(model, test_loader):
    X_test, y_test = [], []
    for X_batch, y_batch in test_loader:
        X_test.append(X_batch.numpy())
        y_test.append(y_batch.numpy())
    X_test = np.vstack(X_test)
    y_test = np.hstack(y_test)

    y_test_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)
    print(f'Test Accuracy: {test_acc:.4f}')

evaluate_naive_bayes(nb_model, test_loader)


Validation Accuracy: 0.5656
Test Accuracy: 0.5657


In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Evaluate Naive Bayes Model
def evaluate_naive_bayes(model, test_loader):
    X_test, y_test = [], []
    for X_batch, y_batch in test_loader:
        X_test.append(X_batch.numpy())
        y_test.append(y_batch.numpy())
    X_test = np.vstack(X_test)
    y_test = np.hstack(y_test)

    y_test_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)
    print(f'Test Accuracy: {test_acc:.4f}')
    return y_test, y_test_pred

y_test, y_test_pred = evaluate_naive_bayes(nb_model, test_loader)

# Calculate additional performance metrics
print("Classification Report:")
print(classification_report(y_test, y_test_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

Test Accuracy: 0.5653
Classification Report:
              precision    recall  f1-score   support

         0.0       0.70      0.44      0.54      4561
         1.0       0.49      0.74      0.59      3283

    accuracy                           0.57      7844
   macro avg       0.59      0.59      0.56      7844
weighted avg       0.61      0.57      0.56      7844

Confusion Matrix:
[[1997 2564]
 [ 846 2437]]


In [24]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np

# Number of folds
k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Store performance metrics
accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

# Convert data from DataLoader to NumPy arrays
X_np, y_np = [], []
for X_batch, y_batch in test_loader:
    X_np.append(X_batch.numpy())
    y_np.append(y_batch.numpy())
X_np = np.vstack(X_np)
y_np = np.hstack(y_np)

# Perform K-Fold Cross-Validation
for train_index, test_index in kf.split(X_np, y_np):
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]

    # Train Naïve Bayes model
    nb_model.fit(X_train, y_train)

    # Make predictions
    y_pred = nb_model.predict(X_test)

    # Compute metrics
    accuracy_list.append(accuracy_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    f1_list.append(f1_score(y_test, y_pred))

# Print average results
print(f"Naïve Bayes Model - {k}-Fold Cross-Validation Results:")
print(f"Average Accuracy: {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}")
print(f"Average Precision: {np.mean(precision_list):.4f} ± {np.std(precision_list):.4f}")
print(f"Average Recall: {np.mean(recall_list):.4f} ± {np.std(recall_list):.4f}")
print(f"Average F1-score: {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}")

from sklearn.metrics import confusion_matrix

# Display Final Evaluation Metrics
print("\n--- Naïve Bayes Model - Final K-Fold Performance ---")
print(f"Mean Accuracy: {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}")
print(f"Mean Precision: {np.mean(precision_list):.4f} ± {np.std(precision_list):.4f}")
print(f"Mean Recall: {np.mean(recall_list):.4f} ± {np.std(recall_list):.4f}")
print(f"Mean F1-score: {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}")

# Get final predictions on the last test set for confusion matrix
y_pred_final = nb_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_final)

# Display Confusion Matrix
print("\nConfusion Matrix:")
print(cm)



Naïve Bayes Model - 10-Fold Cross-Validation Results:
Average Accuracy: 0.6752 ± 0.0152
Average Precision: 0.6809 ± 0.0338
Average Recall: 0.4204 ± 0.0578
Average F1-score: 0.5170 ± 0.0411

--- Naïve Bayes Model - Final K-Fold Performance ---
Mean Accuracy: 0.6752 ± 0.0152
Mean Precision: 0.6809 ± 0.0338
Mean Recall: 0.4204 ± 0.0578
Mean F1-score: 0.5170 ± 0.0411

Confusion Matrix:
[[399  58]
 [196 131]]


In [23]:
from sklearn.metrics import confusion_matrix

# Initialize lists to collect all predictions and true labels across folds
all_y_true = []
all_y_pred = []

# Perform K-Fold Cross-Validation and collect results
for train_index, test_index in kf.split(X_np, y_np):
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]

    # Train Naïve Bayes model
    nb_model.fit(X_train, y_train)

    # Make predictions
    y_pred = nb_model.predict(X_test)

    # Store predictions and true values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

# Compute overall confusion matrix
overall_cm = confusion_matrix(all_y_true, all_y_pred)

# Display Confusion Matrix
print("\nOverall Confusion Matrix Across All K-Folds:")
print(overall_cm)



Overall Confusion Matrix Across All K-Folds:
[[3921  652]
 [1896 1375]]


In [25]:
data

,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,pain,seq_num,icd_code,icd_version,Stroke_Y/N,gender,anchor_age,anchor_year,anchor_year_group
0,98.300000,74.000000,19.000000,97.000000,153.000000,67.000000,-1,70,2,262,10,0,1,46,2196,0
1,98.047064,82.000000,16.000000,96.000000,115.000000,102.000000,-1,-1,1,58,9,1,1,68,2176,1
2,98.000000,109.000000,18.000000,98.000000,113.000000,77.000000,-1,84,2,471,9,0,0,51,2139,1
3,99.900000,74.000000,16.000000,100.000000,151.000000,76.000000,-1,77,1,388,10,0,0,81,2111,1
4,98.400000,83.000000,17.814805,94.000000,142.000000,81.000000,-1,-1,1,58,9,1,1,55,2170,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52285,98.100000,62.000000,16.000000,100.000000,121.000000,62.000000,-1,119,2,330,10,1,0,52,2170,2
52286,98.047064,80.517452,17.814805,97.903309,129.293072,73.612983,-1,92,1,403,10,0,1,25,2185,0
52287,99.800000,92.000000,16.000000,96.000000,160.000000,80.000000,-1,304,1,34,9,0,1,46,2156,0
52288,97.900000,63.000000,20.000000,99.000000,149.000000,97.000000,-1,-1,3,462,9,1,1,61,2141,0


In [26]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Define target variable
target_column = "Stroke_Y/N"
X = data.drop(columns=[target_column]).values
y = data[target_column].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Split dataset into train, validation, and test sets
train_size = int(0.7 * len(X_tensor))
val_size = int(0.15 * len(X_tensor))
test_size = len(X_tensor) - train_size - val_size
train_data, val_data, test_data = random_split(TensorDataset(X_tensor, y_tensor), [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Train SVM Model
def train_svm(train_loader, val_loader):
    X_train, y_train = [], []
    for X_batch, y_batch in train_loader:
        X_train.append(X_batch.numpy())
        y_train.append(y_batch.numpy())
    X_train = np.vstack(X_train)
    y_train = np.hstack(y_train)

    model = SVC(kernel='rbf', probability=True)
    model.fit(X_train, y_train)

    # Validation Accuracy
    X_val, y_val = [], []
    for X_batch, y_batch in val_loader:
        X_val.append(X_batch.numpy())
        y_val.append(y_batch.numpy())
    X_val = np.vstack(X_val)
    y_val = np.hstack(y_val)

    y_val_pred = model.predict(X_val)
    val_acc = accuracy_score(y_val, y_val_pred)
    print(f'Validation Accuracy: {val_acc:.4f}')
    return model

svm_model = train_svm(train_loader, val_loader)

# Evaluate SVM Model
def evaluate_svm(model, test_loader):
    X_test, y_test = [], []
    for X_batch, y_batch in test_loader:
        X_test.append(X_batch.numpy())
        y_test.append(y_batch.numpy())
    X_test = np.vstack(X_test)
    y_test = np.hstack(y_test)

    y_test_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)
    print(f'Test Accuracy: {test_acc:.4f}')
    return y_test, y_test_pred

y_test, y_test_pred = evaluate_svm(svm_model, test_loader)

# Calculate additional performance metrics
print("Classification Report:")
print(classification_report(y_test, y_test_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


Validation Accuracy: 0.6594
Test Accuracy: 0.6539
Classification Report:
              precision    recall  f1-score   support

         0.0       0.68      0.77      0.72      4597
         1.0       0.60      0.49      0.54      3247

    accuracy                           0.65      7844
   macro avg       0.64      0.63      0.63      7844
weighted avg       0.65      0.65      0.65      7844

Confusion Matrix:
[[3546 1051]
 [1664 1583]]


In [27]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Define number of folds
K = 5
kf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

# Initialize lists to store performance metrics across folds
accuracy_list, precision_list, recall_list, f1_list = [], [], [], []
all_y_true, all_y_pred = [], []

# Perform K-Fold Cross-Validation
for train_index, test_index in kf.split(X_np, y_np):
    # Split data into training and testing for this fold
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]

    # Train SVM model
    svm_model = SVC(kernel='rbf', C=1.0, probability=True)  # RBF kernel, tune hyperparameters as needed
    svm_model.fit(X_train, y_train)

    # Make predictions
    y_pred = svm_model.predict(X_test)

    # Store true labels and predictions for confusion matrix
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    # Compute performance metrics for this fold
    accuracy_list.append(accuracy_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    f1_list.append(f1_score(y_test, y_pred))

# Compute overall confusion matrix
overall_cm = confusion_matrix(all_y_true, all_y_pred)

# Display Final Evaluation Metrics
print("\n--- SVM Model - Final K-Fold Performance ---")
print(f"Mean Accuracy: {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}")
print(f"Mean Precision: {np.mean(precision_list):.4f} ± {np.std(precision_list):.4f}")
print(f"Mean Recall: {np.mean(recall_list):.4f} ± {np.std(recall_list):.4f}")
print(f"Mean F1-score: {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}")

# Display Confusion Matrix
print("\nOverall Confusion Matrix Across All K-Folds:")
print(overall_cm)


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- SVM Model - Final K-Fold Performance ---
Mean Accuracy: 0.5829 ± 0.0003
Mean Precision: 0.0000 ± 0.0000
Mean Recall: 0.0000 ± 0.0000
Mean F1-score: 0.0000 ± 0.0000

Overall Confusion Matrix Across All K-Folds:
[[4572    1]
 [3271    0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
